In [ ]:
import os
import numpy as np
import pandas as pd
import re
from langchain import PromptTemplate
from langchain_community.llms import Ollama
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from pprint import pprint
import logging
from datetime import datetime, timedelta
import ast
import json
import time

In [ ]:
def get_response_llm(tweet1, tweet2, topic, topic_categories, model, index):
    print(index)
    response = {
        'tweet1_stance_explanation': '',
        'tweet1_stance': '',        
        'tweets_agreement_explanation': '',
        'tweets_agreement': '',        
        'tweet1_affect_explanation': '',
        'tweet1_affect': '',                
    }     
    if tweet1 and tweet2:
        print('in if')
        model = model.bind(
            functions=[
                {
                    "name": "get_affective_polarization",
                    "description": (
                        "Analyze the content of the provided tweets to assess their stance and emotional tone with respect to the topic: " + topic + 
                        ". Provide detailed classifications for stance, agreement between tweets, and affective polarization (emotional negativity towards opposing views)."
                    ),
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "tweet1_stance_explanation": {
                                "type": "string",
                                "description": (
                                    "Provide a brief explanation of tweet1's stance on the topic " + topic + 
                                    ". If tweet1 supports " + topic_categories[0] + 
                                    ", or opposes with " + topic_categories[1] + 
                                    ", explain the reasoning. If the stance is unclear, label it as " + topic_categories[2] + "."
                                )
                            },
                            "tweet1_stance": {
                                "type": "string",
                                "description": (
                                    "Classify tweet1's stance on the topic: " + topic + 
                                    ". Possible values: " + ", ".join(topic_categories) + 
                                    ". This classification should be based on the explanation provided in 'tweet1_stance_explanation'."
                                ),
                                "enum": topic_categories
                            },
                            "tweets_agreement_explanation": {
                                "type": "string",
                                "description": (
                                    "Provide an explanation of whether tweet1 and tweet2 agree or disagree on the topic: " + topic + 
                                    ". Agreement indicates similar views; disagreement means opposing views. If tweet2 is not available, state 'not applicable'. If the agreement is unclear, provide reasoning."
                                )
                            },
                            "tweets_agreement": {
                                "type": "string",
                                "description": (
                                    "Classify the agreement between tweet1 and tweet2 with respect to the topic: " + topic + 
                                    ". Possible values: yes (agreement), no (disagreement), don't know (unclear)."
                                ),
                                "enum": ["yes", "no", "don't know"]
                            },
                            "tweet1_affect_explanation": {
                                "type": "string",
                                "description": (
                                    "Explain whether tweet1 contains deeply negative emotions or attitudes specifically towards people who hold opposing views on the topic: " + topic + 
                                    ". The focus is on emotional negativity beyond the stance itself."
                                )
                            },
                            "tweet1_affect": {
                                "type": "string",
                                "description": (
                                    "Classify tweet1's affective polarization, i.e., emotional negativity specifically towards opposing views on the topic: " + topic + 
                                    ". Possible values: yes (contains affective polarization), no (doesn't contain), don't know (uncertain)."
                                ),
                                "enum": ["yes", "no", "don't know"]
                            },
                        },
                        "required": [
                            "tweet1_stance_explanation", "tweet1_stance", 
                            "tweets_agreement_explanation", "tweets_agreement",
                            "tweet1_affect_explanation", "tweet1_affect",
                        ],
                    }
                }
            ],
            function_call={"name": "get_affective_polarization"},
        )    
        res_1, res_2, res_3, res_4, res_5, res_6 = '', '', '', '', '', ''    
        pattern_res_1 = re.compile(r'tweet1_stance_explanation', re.IGNORECASE)
        pattern_res_2 = re.compile(r'tweet1_stance', re.IGNORECASE)
        pattern_res_3 = re.compile(r'tweets_agreement_explanation', re.IGNORECASE)
        pattern_res_4 = re.compile(r'tweets_agreement', re.IGNORECASE)
        pattern_res_5 = re.compile(r'tweet1_affect_explanation', re.IGNORECASE)
        pattern_res_6 = re.compile(r'tweet1_affect', re.IGNORECASE)    
        try:
            result = model.invoke("Tweet1: " + tweet1 + " \n\n\n\n Tweet2: " + tweet2)
            tool_calls = result.to_json()['kwargs']['tool_calls']
            for tc in tool_calls:
                if tc['name'] == 'get_affective_polarization':
                    args = tc['args']
                    for key, value in args.items():
                        if pattern_res_1.search(key):
                            res_1 = value
                        elif pattern_res_2.search(key) and 'explanation' not in key:
                            res_2 = value
                        elif pattern_res_3.search(key):
                            res_3 = value
                        elif pattern_res_4.search(key) and 'explanation' not in key:
                            res_4 = value
                        elif pattern_res_5.search(key):
                            res_5 = value
                        elif pattern_res_6.search(key) and 'explanation' not in key:
                            res_6 = value
                    break
        except Exception as e1:
            try:
                error_message = str(e1)
                if 'tool_input' in error_message:
                    json_start_index = error_message.find('{')
                    json_end_index = error_message.rfind('}') + 1
                    json_str = error_message[json_start_index:json_end_index]
                    parsed_json = json.loads(json_str)
                    args = parsed_json.get("tool_input", {})
                    for key, value in args.items():
                        if pattern_res_1.search(key):
                            res_1 = value
                        elif pattern_res_2.search(key) and 'explanation' not in key:
                            res_2 = value
                        elif pattern_res_3.search(key):
                            res_3 = value
                        elif pattern_res_4.search(key) and 'explanation' not in key:
                            res_4 = value
                        elif pattern_res_5.search(key):
                            res_5 = value
                        elif pattern_res_6.search(key) and 'explanation' not in key:
                            res_6 = value            
                else:
                    raise Exception('tool_input not in error message!')
            except Exception as e2:
                if "don't have direct access to external" in str(e1) or "don't have direct access to external" in str(e2) or \
                "need to analyze the content" in str(e1) or "need to analyze the content" in str(e2) or \
                "need more information" in str(e1) or "need more information" in str(e2) or \
                "need more context" in str(e1) or "need more context" in str(e2) or \
                "Expecting ',' delimiter" in str(e1) or "Expecting ',' delimiter" in str(e2) or \
                "Error parsing JSON: Invalid" in str(e1) or "Error parsing JSON: Invalid" in str(e2) or \
                "Error parsing JSON: Extra data" in str(e1) or "Error parsing JSON: Extra data" in str(e2) or \
                "not sure how they relate to the topic" in str(e1) or "not sure how they relate to the topic" in str(e2) or \
                "don't have enough information" in str(e1) or "don't have enough information" in str(e2) or \
                "cannot access external" in str(e1) or "cannot access external" in str(e2) or \
                "don't have the capability to access external" in str(e1) or "don't have the capability to access external" in str(e2) or \
                "don't seem to be related to" in str(e1) or "don't seem to be related to" in str(e2):
                    pass
                else:
                    add_log(f'Error in model.invoke(): {e1}\nError parsing JSON: {e2}')                
        response = {
            'tweet1_stance_explanation': res_1,
            'tweet1_stance': res_2,        
            'tweets_agreement_explanation': res_3,
            'tweets_agreement': res_4,        
            'tweet1_affect_explanation': res_5,
            'tweet1_affect': res_6,                
        }            
    elif tweet1:
        print('in elif')
        model = model.bind(
            functions=[
                {
                    "name": "get_affective_polarization",
                    "description": (
                        "Analyze the content of the provided tweet to assess it's stance and emotional tone with respect to the topic: " + topic + 
                        ". Provide detailed classifications for stance, and affective polarization (emotional negativity towards opposing views)."
                    ),
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "tweet1_stance_explanation": {
                                "type": "string",
                                "description": (
                                    "Provide a brief explanation of tweet1's stance on the topic " + topic + 
                                    ". If tweet1 supports " + topic_categories[0] + 
                                    ", or opposes with " + topic_categories[1] + 
                                    ", explain the reasoning. If the stance is unclear, label it as " + topic_categories[2] + "."
                                )
                            },
                            "tweet1_stance": {
                                "type": "string",
                                "description": (
                                    "Classify tweet1's stance on the topic: " + topic + 
                                    ". Possible values: " + ", ".join(topic_categories) + 
                                    ". This classification should be based on the explanation provided in 'tweet1_stance_explanation'."
                                ),
                                "enum": topic_categories
                            },
                            "tweet1_affect_explanation": {
                                "type": "string",
                                "description": (
                                    "Explain whether tweet1 contains deeply negative emotions or attitudes specifically towards people who hold opposing views on the topic: " + topic + 
                                    ". The focus is on emotional negativity beyond the stance itself."
                                )
                            },
                            "tweet1_affect": {
                                "type": "string",
                                "description": (
                                    "Classify tweet1's affective polarization, i.e., emotional negativity specifically towards opposing views on the topic: " + topic + 
                                    ". Possible values: yes (contains affective polarization), no (doesn't contain), don't know (uncertain)."
                                ),
                                "enum": ["yes", "no", "don't know"]
                            },
                        },
                        "required": [
                            "tweet1_stance_explanation", "tweet1_stance", 
                            "tweet1_affect_explanation", "tweet1_affect",
                        ],
                    }
                }
            ],
            function_call={"name": "get_affective_polarization"},
        )    
        res_1, res_2, res_3, res_4, res_5, res_6 = '', '', '', '', '', ''    
        pattern_res_1 = re.compile(r'tweet1_stance_explanation', re.IGNORECASE)
        pattern_res_2 = re.compile(r'tweet1_stance', re.IGNORECASE)
        pattern_res_5 = re.compile(r'tweet1_affect_explanation', re.IGNORECASE)
        pattern_res_6 = re.compile(r'tweet1_affect', re.IGNORECASE)    
        try:
            print('trying')
            result = model.invoke("Tweet1: " + tweet1)
            print(result)
            tool_calls = result.to_json()['kwargs']['tool_calls']
            for tc in tool_calls:
                if tc['name'] == 'get_affective_polarization':
                    args = tc['args']
                    for key, value in args.items():
                        if pattern_res_1.search(key):
                            res_1 = value
                        elif pattern_res_2.search(key) and 'explanation' not in key:
                            res_2 = value
                        elif pattern_res_5.search(key):
                            res_5 = value
                        elif pattern_res_6.search(key) and 'explanation' not in key:
                            res_6 = value
                    break
        except Exception as e1:
            print(e1)
            try:
                error_message = str(e1)
                if 'tool_input' in error_message:
                    json_start_index = error_message.find('{')
                    json_end_index = error_message.rfind('}') + 1
                    json_str = error_message[json_start_index:json_end_index]
                    parsed_json = json.loads(json_str)
                    args = parsed_json.get("tool_input", {})
                    for key, value in args.items():
                        if pattern_res_1.search(key):
                            res_1 = value
                        elif pattern_res_2.search(key) and 'explanation' not in key:
                            res_2 = value
                        elif pattern_res_5.search(key):
                            res_5 = value
                        elif pattern_res_6.search(key) and 'explanation' not in key:
                            res_6 = value            
                else:
                    raise Exception('tool_input not in error message!')
            except Exception as e2:
                if "don't have direct access to external" in str(e1) or "don't have direct access to external" in str(e2) or \
                "need to analyze the content" in str(e1) or "need to analyze the content" in str(e2) or \
                "need more information" in str(e1) or "need more information" in str(e2) or \
                "need more context" in str(e1) or "need more context" in str(e2) or \
                "Expecting ',' delimiter" in str(e1) or "Expecting ',' delimiter" in str(e2) or \
                "Error parsing JSON: Invalid" in str(e1) or "Error parsing JSON: Invalid" in str(e2) or \
                "Error parsing JSON: Extra data" in str(e1) or "Error parsing JSON: Extra data" in str(e2) or \
                "not sure how they relate to the topic" in str(e1) or "not sure how they relate to the topic" in str(e2) or \
                "don't have enough information" in str(e1) or "don't have enough information" in str(e2) or \
                "cannot access external" in str(e1) or "cannot access external" in str(e2) or \
                "don't have the capability to access external" in str(e1) or "don't have the capability to access external" in str(e2) or \
                "don't seem to be related to" in str(e1) or "don't seem to be related to" in str(e2):
                    pass
                else:
                    add_log(f'Error in model.invoke(): {e1}\nError parsing JSON: {e2}')                
        response = {
            'tweet1_stance_explanation': res_1,
            'tweet1_stance': res_2,        
            'tweets_agreement_explanation': res_3,
            'tweets_agreement': res_4,        
            'tweet1_affect_explanation': res_5,
            'tweet1_affect': res_6,                
        }            
    else:
        pass
    return response
        

def generate_info_SAA(df, llm, topic, topic_categories):    
    df["Tweet Stance"] = ""
    df["Tweet Affect"] = ""
    df["Tweets Agreement"] = ""

    df["Tweet Stance Explanation"] = ""
    df["Tweet Affect Explanation"] = ""
    df["Tweets Agreement Explanation"] = ""

    for i in range(df.shape[0]):
        id_tweet = df.index[i]
        txt_tweet = df.loc[id_tweet]["Tweet Text"]
        id_parent_tweet = df.loc[id_tweet]["Parent Tweet ID"]

        if pd.api.types.is_numeric_dtype(type(id_parent_tweet)):
            is_nan = np.isnan(id_parent_tweet)
        else:
            is_nan = False

        if id_parent_tweet and not is_nan and id_parent_tweet != "Root Author":
            try:
                id_parent_tweet = int(id_parent_tweet)                
                
                txt_parent_tweet = df.loc[id_parent_tweet]["Tweet Text"]

                result =  get_response_llm(txt_tweet, txt_parent_tweet, topic, topic_categories, llm, i) 

                df.at[id_tweet, "Tweet Stance"] = result["tweet1_stance"]
                df.at[id_tweet, "Tweet Affect"] = result["tweet1_affect"]
                df.at[id_tweet, "Tweets Agreement"] = result["tweets_agreement"]

                df.at[id_tweet, "Tweet Stance Explanation"] = result["tweet1_stance_explanation"]
                df.at[id_tweet, "Tweet Affect Explanation"] = result["tweet1_affect_explanation"]
                df.at[id_tweet, "Tweets Agreement Explanation"] = result["tweets_agreement_explanation"]   

            except Exception as e:
                add_log(f'Error in SAA: {e}')                

                result =  get_response_llm(txt_tweet, '', topic, topic_categories, llm, i) 

                df.at[id_tweet, "Tweet Stance"] = result["tweet1_stance"]
                df.at[id_tweet, "Tweet Affect"] = result["tweet1_affect"]

                df.at[id_tweet, "Tweet Stance Explanation"] = result["tweet1_stance_explanation"]
                df.at[id_tweet, "Tweet Affect Explanation"] = result["tweet1_affect_explanation"]
        else:

            result =  get_response_llm(txt_tweet, '', topic, topic_categories, llm, i) 

            df.at[id_tweet, "Tweet Stance"] = result["tweet1_stance"]
            df.at[id_tweet, "Tweet Affect"] = result["tweet1_affect"]

            df.at[id_tweet, "Tweet Stance Explanation"] = result["tweet1_stance_explanation"]
            df.at[id_tweet, "Tweet Affect Explanation"] = result["tweet1_affect_explanation"]
        
    return df

def classify_interaction(stance_tweet, stance_parent_tweet, affect_tweet, affect_parent_tweet, tweets_agreement):
    # Function to classify the interaction
    # Generate categories for pairs of tweets
    #
    # Tweet Stance |  Parent Stance | Tweet Affect | Parent Affect | Agreement | Class | Description 
    #      same stance              |         yes anywhere         |     yes   |   10  | very high danger - eco chamber like no interaction with the oposite stance but negative emotions towards the oposite stance        
    #      same stance              |         yes anywhere         |     no    |    8  | high danger - disagreement on the same stance is a good sign but still negative emotions towards the oposite stance
    #      same stance              |              no              |     yes   |    6  | medium danger - eco chamber like no interaction with the oposite stance        
    #      same stance              |              no              |     no    |    2  | low danger - disagreement on the same stance is very good 
    #      opposite stance          |         yes anywhere         |     no    |    8  | high danger - disagreement opposite stance but with negative emotions (still a bit better that there is interaction)
    #      opposite stance          |         yes anywhere         |     yes   |    6  | medium danger - agreeemnt opposite stances is good but with negative emotions
    #      opposite stance          |              no              |     no    |    4  | less danger - interaction with no negative emotions
    #      opposite stance          |              no              |     yes   |    0  | no danger - interaction, no negative emotions, and agreement

    #
    # TODO: further refinement - understand the implications on the replies
    #
    # if the Parent Affect = yes and Tweet Affect = no but Agreement then this is more dangerous than
    # if the Parent Affect = no and Tweet Affect = yes but Agreement 
    
    
    same_stance = stance_tweet == stance_parent_tweet
    any_affect_yes = affect_tweet == 'yes' or affect_parent_tweet == 'yes'

    # Correct classification logic based on the detailed table
    if same_stance:
        if any_affect_yes:
            if tweets_agreement == 'yes':
                return 10  # very high danger - eco chamber like no interaction with the oposite stance but negative emotions towards the oposite stance    
            else:
                return 8  # high danger - disagreement on the same stance is a good sign but still negative emotions towards the oposite stance
        else:
            if tweets_agreement == 'yes':
                return 6  # medium danger - eco chamber like no interaction with the oposite stance    
            else:
                return 2  # low danger - disagreement on the same stance is very good 
    else:  # Opposite stance
        if any_affect_yes:
            if tweets_agreement == 'yes':
                return  6 # medium danger - agreeemnt opposite stances is good but with negative emotions
            else:
                return 8  # high danger - disagreement opposite stance but with negative emotions (still a bit better that there is interaction)
        else:
            if tweets_agreement == 'yes':
                return 0  # no danger - interaction, no negative emotions, and agreement
            else:
                return 4  # less danger - interaction with no negative emotions

def generate_info_IC(df):
    
    df['Interaction Class'] = None

    for i in range(df.shape[0]):
        id_tweet = df.index[i]
        txt_tweet = df.loc[id_tweet]["Tweet Text"]
        id_parent_tweet = df.loc[id_tweet]["Parent Tweet ID"]
        stance_tweet = df.loc[id_tweet]["Tweet Stance"]
        affect_tweet = df.loc[id_tweet]["Tweet Affect"]
        tweets_agreement = df.loc[id_tweet]["Tweets Agreement"]

        if pd.api.types.is_numeric_dtype(type(id_parent_tweet)):
            is_nan = np.isnan(id_parent_tweet)
        else:
            is_nan = False

        if id_parent_tweet and not is_nan and id_parent_tweet != "Root Author":
            try:
                id_parent_tweet = int(id_parent_tweet)                

                txt_parent_tweet = df.loc[id_parent_tweet]["Tweet Text"]
                stance_parent_tweet = df.loc[id_parent_tweet]["Tweet Stance"]
                affect_parent_tweet = df.loc[id_parent_tweet]["Tweet Affect"]
                interaction_class = classify_interaction(stance_tweet, stance_parent_tweet, affect_tweet, affect_parent_tweet, tweets_agreement)
                df.at[id_tweet, "Interaction Class"] = interaction_class        
            except Exception as e:
                add_log(f'Error in IC: {e}')                                
                continue
    return df

def add_log(message, level='info'):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"Time: {current_time} - {message}"
    
    if level == 'info':
        logging.info(log_message)
    elif level == 'error':
        logging.error(log_message)
    else:
        logging.debug(log_message)  
        

In [ ]:
tweet1 = "The science is clear. Climate change is real, and urgent action is needed before it's too late!" ## original tweet that has no parent tweet
tweet2 = ""
topic = "climate change"
topic_categories = ["belief", "disbelief", "don't know"]
index = 0
model = OllamaFunctions(model="llama3.1:70b", temperature=0.0)
response = get_response_llm(tweet1, tweet2, topic, topic_categories, model, index)
print(response)

In [4]:
tweet1 = "Climate change is a hoax pushed by the corrupt elites to brainwash the masses. Wake up, fool!" ## reply tweet that has parent tweet (the original tweet)
tweet2 = "The science is clear. Climate change is real, and urgent action is needed before it's too late!" ## parent tweet
topic = "climate change"
topic_categories = ["belief", "disbelief", "don't know"]
index = 1
model = OllamaFunctions(model="llama3.1:70b", temperature=0.0)
response = get_response_llm(tweet1, tweet2, topic, topic_categories, model, index)
print(response)

1
in if
{'tweet1_stance_explanation': 'Tweet1 expresses disbelief in climate change, calling it a hoax and implying that those who believe in it are brainwashed. The tone is confrontational and dismissive.', 'tweet1_stance': 'disbelief', 'tweets_agreement_explanation': 'Tweet1 and Tweet2 have opposing views on climate change. Tweet1 denies its existence, while Tweet2 affirms it and calls for urgent action. They clearly disagree on the topic.', 'tweets_agreement': 'no', 'tweet1_affect_explanation': "Tweet1 contains deeply negative emotions towards people who hold opposing views on climate change, calling them 'fool' and implying they are brainwashed. This language is emotionally charged and divisive.", 'tweet1_affect': 'yes'}
